Exploratory analysis of the review data to find:
* What is the size distribution as well as the mean/median size of the reviews.

In [1]:
import pyodbc
dwh_login = {'option': '3',
             'database': 'datawarehouse',
             'driver': '{PostgreSQL ANSI}',
             'server': 'je-dw.cspauv1pfeh3.eu-west-1.redshift.amazonaws.com',
             'pwd': '',
             'user': '',
             'port': '5439'}

sql = 'select 1;'

try:
    schema_conn = pyodbc.connect('DSN=Redshift x64')
    schema_db = schema_conn.cursor()
    data = schema_db.execute(sql).fetchall()
    schema_conn.commit()
#    schema_db.close()
#    schema_conn.close()
except Exception, ex:
    raise

print data

[(1, )]


In [10]:
#What is the number of customer reviews per year in the ecom.rating table
#%matplotlib inline
from scipy.stats import kde
import pandas as pd
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
q1="""
select commentary as review from ecom.rating where commentary<>'' and country_key=8
ORDER BY RANDOM()
limit 10000
"""
#function that returns the vector of values (lentgh of each string/review) their mean and their std.
def getDescriptive(df):
    vectorD=[]
    for i in range(len(df)):
        vectorD.append(len(df["review"][i]))
    return vectorD, np.mean(vectorD), np.median(vectorD), np.std(vectorD)    

df1 = pd.read_sql(q1, schema_conn)
vector1, mean1, median1, std1 = getDescriptive(df1)

print "units: number of characters in a review"
print "----------------------------------------"
print "mean: ", mean1
print "median", median1
print "std", std1
print "----------------------------------------"

### plotting (review length) ###
#kernel density stimation (non-parametric prob distrib)
NN=200
x1=np.asarray(vector1)
density1 = kde.gaussian_kde(x1)
xgrid1 = np.linspace(x1.min(), x1.max(), NN)

plt.title("Probability distribution review lengths",fontsize=22)
plt.xlabel("Length of the review (in characters)",fontsize=18)
plt.ylabel("Prob. Density Function",fontsize=18)
plt.xlim([0,500])
plt.tick_params(axis='both', which='major', labelsize=16)
plt.plot(xgrid1, density1(xgrid1), 'r-',linewidth=2.0)
plt.plot(x1, np.zeros(x1.shape), 'r+', ms=12) #rug plot
plt.show()

units: number of characters in a review
----------------------------------------
mean:  113.6434
median 72.0
std 132.521755333
----------------------------------------
